In [1]:
import os

os.chdir("../")
%pwd

'c:\\Users\\hasan\\End to end\\FullMLPipe'

In [2]:
from src.FullMLPipe.config.configuration import ConfigurationManager

config_manager = ConfigurationManager()

secrets = config_manager.get_secret()

2025-08-24 15:36:20,028: INFO: common: yaml file: config\config.yaml loaded successfully
2025-08-24 15:36:20,030: INFO: common: yaml file: params.yaml loaded successfully
2025-08-24 15:36:20,032: INFO: common: yaml file: schema.yaml loaded successfully
2025-08-24 15:36:20,033: INFO: common: created directory at: artifacts


In [3]:
os.environ["MLFLOW_TRACKING_URI"] = secrets.mlflow_uri
os.environ["MLFLOW_TRACKING_USERNAME"] = secrets.mlflow_username
os.environ["MLFLOW_TRACKING_PASSWORD"] = secrets.mlflow_password

In [4]:
from dataclasses import dataclass
from pathlib import Path 

In [ ]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [6]:
from src.FullMLPipe.constants import *
from src.FullMLPipe.utils.common import *

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        target = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        return ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metric_file_name= config.metric_file_name,
            target_column= target.name,
            mlflow_uri= secrets.mlflow_uri
        )

In [8]:
import os 
import pandas as pd 
from sklearn.metrics import mean_squared_error , mean_absolute_error , r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np 
import joblib

c:\Users\hasan\End to end\FullMLPipe\venv\lib\site-packages\pydantic\_internal\_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [9]:
class ModelEvaluation:
    def __init__(self , config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        
        return rmse , mae , r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        X_test = test_data.drop([self.config.target_column] , axis=1)
        y_test = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(X_test)
            
            (rmse,mae,r2) = self.eval_metrics(y_test , predicted_qualities)
            
            scores = {"rmse" : rmse, "mae": mae , "r2" : r2}
            save_json(path=Path(self.config.metric_file_name) , data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse" , rmse)
            mlflow.log_metric("mae" , mae)
            mlflow.log_metric("r2" , r2)
            
            if tracking_url_type_store != 'file':
                mlflow.sklearn.log_model(model, "model")
            else:
                mlflow.sklearn.log_model(model, "model")


In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config = model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

2025-08-24 15:36:22,104: INFO: common: yaml file: config\config.yaml loaded successfully
2025-08-24 15:36:22,105: INFO: common: yaml file: params.yaml loaded successfully
2025-08-24 15:36:22,108: INFO: common: yaml file: schema.yaml loaded successfully
2025-08-24 15:36:22,109: INFO: common: created directory at: artifacts
2025-08-24 15:36:22,110: INFO: common: created directory at: artifacts/model_evaluation
2025-08-24 15:36:22,838: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json
